# Imports 
Necessary imports, based in part on [this](https://www.kaggle.com/code/awsaf49/planttraits2024-kerascv-starter-notebook)

In [7]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch

import keras_cv
import keras
from keras import ops
import tensorflow as tf

import cv2
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import joblib

import matplotlib.pyplot as pl

In [2]:
import model

nn = model.Model("model1")

print(nn.name)

model1


# Data pre-processing

## Loading

In [3]:
# print cwd 
print(os.getcwd())

/home/applesauce/vu-ml-project/src


## Visualizations